In [1]:
import os
import json
from tqdm import tqdm

import datasets
from transformers import pipeline

In [2]:
test = datasets.load_from_disk("data/test_dataset")
test

Dataset({
    features: ['src', 'lang', 'text'],
    num_rows: 12013630
})

In [3]:
batch_size = 512

In [4]:
model = pipeline("text-classification", "data/results/checkpoint-76000", truncation=True, padding="max_length", batch_size=batch_size)

Device set to use cuda:0


In [5]:
examples = test[:5]
print(examples)
preds = model(examples["text"])
print(preds)

{'src': ['mt560', 'mt560', 'mt560', 'mt560', 'lti'], 'lang': ['ces_Latn', 'mlt_Latn', 'ayr_Latn', 'kmb_Latn', 'uig_Arab'], 'text': ['Co kdybychom dokázali vytvořit struktury, které by byly doplňkem živé hmoty?', "L-Artikolu 8 (1) (a) (ii) tar-Regolament (KE) Nru 998 / 2003 jipprevedi li, f'każijiet fejn il-pets jittieħdu minn ċerti pajjiżi terzi lejn l-Irlanda, Malta, l-Isvezja u r-Renju Unit, għandhom japplikaw il-ħtiġiet addizzjonali previsti fl-Artikolu 6 ta 'dak ir-Regolament.", '"Cristo" aruxa kjristós uka griego arutwa juti, hebreo arutxa maschíaj (mesías) sasirakispawa.', 'Mu jimbulule, mukonda diahi o ku bhanga o kikalakalu kiki, ki suínisa dingi o kikalakalu kia ku boka o njimbu ia Utuminu.', 'بۇلارنىڭ ھەممىسى يات ئەللىك خوتۇنلارنى ئالغانلار ئىدى، ۋە شۇنىڭدەك ئۇلاردىن بەزىلەرنىڭ خوتۇنلىرى پەرزەنتمۇ كۆرگەنىدى.']}
[{'label': 'ces_Latn', 'score': 0.9999101161956787}, {'label': 'mlt_Latn', 'score': 0.9997753500938416}, {'label': 'ayr_Latn', 'score': 0.9823504686355591}, {'label': 

In [6]:
gt_labels = []
preds_labels = []
for i, examples in tqdm(enumerate(test.iter(batch_size=batch_size))):
	preds = model(examples["text"])
	gt_labels.append(examples["lang"])
	preds_labels.append(preds)
	
	if i % 100 == 0:
		with open("data/predictions/save.json", "w") as file:
			json.dump({"prediction": preds_labels, "expected": gt_labels}, file)

9it [00:05,  1.56it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
1500it [16:45,  1.49it/s]


KeyboardInterrupt: 